<a href="https://colab.research.google.com/github/agnxsh/task-specific-hf/blob/main/task_specific_hf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Adding a task specific head on top of the BERT model, for dealing with custom NLP tasks

###Apart from the last layer, which is the Model head, a layer which is task specific, apart from the other shared layers that are task agnostic, these layers mainly consists of transformer layers and token embeddings.

#### The base model in HuggingFace is usually represented as a python class depicting the BERT class or the GPT-2 class, and what it returns is usually the last hidden layer of the transformer, this is represented in the following way:

output[0] => this is the last hidden state

####In the custom layer for HF, we pass this last hidden state as the input to our new task-specific layer

##Practically speaking the base model has to support the task that we want to add, for example we can't just use DistilBERT for machine translation, because it does not support the same, however DistilBERT can be used for Question-Answering tasks.

In [1]:
# Here we're using a Kaggle Dataset which consists of News Headlines for Sarcasm Detection

In [6]:
!pip install datasets
!pip install transformers
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
import numpy as np
import pandas as pd

from datasets import load_dataset,DatasetDict
from transformers import DataCollatorWithPadding, AutoModelForSequenceClassification,Trainer,TrainingArguments,AutoTokenizer, AutoModel,AutoConfig
from transformers.modeling_outputs import TokenClassifierOutput
import torch
import torch.nn as nn
import os
for dirname, _ ,filenames in os.walk("/content/input"):
  for filename in filenames:
    print(os.path.join(dirname,filename))


/content/input/Sarcasm_Headlines_Dataset.json
/content/input/Sarcasm_Headlines_Dataset_v2.json


In [13]:
dataset_v2_path = "/content/input/Sarcasm_Headlines_Dataset_v2.json"

In [14]:
df = pd.read_json(dataset_v2_path, lines=True)
df.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


#Load Dataset with HF's load_dataset

In [15]:
dataset_hf = load_dataset("json",data_files = dataset_v2_path)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-5999052152860cf8/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]